In [32]:
import sqlite3
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import statsmodels.api as sm_api

%matplotlib inline

db_file = 'elderspeak_detect.db'
con = sqlite3.connect(db_file)

df = pd.read_sql_query('''
    select	
        td.elderspeak,
		pr.spraaksnelheid,
		pr.geluidsniveau,
       	pr.toonhoogte,
        tr.cilt,
        tr.woordlengteratio,
        tr.aantal_collectieve_voornaamwoorden,
        tr.aantal_verkleinwoorden,
        tr.aantal_herhalingen,
        tr.textcat_elderspeak_score,
        id.geslacht,
        id.leeftijd,
        id.moedertaal,
        id.student_zorg,
        id.werk_zorg
    from praat_resultaten pr
    join tekst_resultaten tr on pr.audio_id = tr.audio_id
    join teksten t on t.tekst_id = tr.tekst_id
    join test_data td on td.audio_id = tr.audio_id
    left join input_data id on tr.audio_id in (id.leeftijdsgenoot_opname, id.oudere_opname) 
    where t.methode = 'GOOGLE_ENKEL_NL_BE'
    and pr.spraaksnelheid > 0''', con)
con.close()

print(f"Aantal kolommen: { len(df.columns) }")
print(f"Aantal rijen: { len(df.index) }")
df.tail()

Aantal kolommen: 15
Aantal rijen: 47


,elderspeak,spraaksnelheid,geluidsniveau,toonhoogte,cilt,woordlengteratio,aantal_collectieve_voornaamwoorden,aantal_verkleinwoorden,aantal_herhalingen,textcat_elderspeak_score,geslacht,leeftijd,moedertaal,student_zorg,werk_zorg
42,0,3.159722,73.272019,111.466943,55.39,0.166667,0,4,1,7.054659e-10,M,40,nl,0,0
43,1,2.892109,71.059838,194.761772,65.78,0.114754,2,3,7,1.000000e+00,V,55,nl,1,1
44,0,3.540215,75.012906,129.553227,68.45,0.044944,2,4,4,2.404300e-06,M,40,nl,0,0
45,0,2.429387,33.696677,98.056544,-1.00,-1.000000,0,0,0,0.000000e+00,M,18,nl,0,0
46,1,1.281776,66.549338,237.226846,65.39,0.111111,0,0,1,1.000000e+00,V,73,nl,0,0


---

In [33]:
reg = sm.ols(formula="elderspeak ~ spraaksnelheid + geluidsniveau + toonhoogte", data=df).fit()
reg.params

Intercept        -0.682542
spraaksnelheid   -0.154075
geluidsniveau     0.016376
toonhoogte        0.002186
dtype: float64

In [34]:
reg = sm.ols(formula="elderspeak ~ cilt + woordlengteratio + aantal_collectieve_voornaamwoorden + aantal_verkleinwoorden + aantal_herhalingen", data=df).fit()
reg.params

Intercept                             0.641273
cilt                                 -0.004738
woordlengteratio                      0.603489
aantal_collectieve_voornaamwoorden   -0.045982
aantal_verkleinwoorden               -0.033458
aantal_herhalingen                    0.070305
dtype: float64

In [35]:
reg = sm.ols(formula="elderspeak ~ spraaksnelheid + geluidsniveau + toonhoogte + cilt + woordlengteratio + aantal_collectieve_voornaamwoorden + aantal_verkleinwoorden + aantal_herhalingen + textcat_elderspeak_score", data=df).fit()
reg.params

Intercept                            -0.338043
spraaksnelheid                       -0.113442
geluidsniveau                         0.011385
toonhoogte                            0.001475
cilt                                 -0.004189
woordlengteratio                     -0.120754
aantal_collectieve_voornaamwoorden   -0.032410
aantal_verkleinwoorden                0.032736
aantal_herhalingen                    0.020968
textcat_elderspeak_score              0.631851
dtype: float64

---

In [36]:
df['reg_model_praat'] = -0.682542 -0.154075 * df['spraaksnelheid'] + 0.016376 * df['geluidsniveau'] + 0.002186 * df['toonhoogte']
df['reg_model_tekst'] = 0.641273 -0.004738 * df['cilt'] + 0.603489 * df['woordlengteratio'] -0.045982 * df['aantal_collectieve_voornaamwoorden'] -0.033458 * df['aantal_verkleinwoorden'] + 0.070305 * df['aantal_herhalingen']
df['reg_model_volledig'] = -0.338043 -0.113442 * df['spraaksnelheid'] + 0.011385 * df['geluidsniveau'] + 0.001475 * df['toonhoogte'] -0.004189 * df['cilt'] -0.120754 * df['woordlengteratio'] -0.032410 * df['aantal_collectieve_voornaamwoorden'] + 0.032736 * df['aantal_verkleinwoorden'] + 0.020968 * df['aantal_herhalingen'] + 0.631851 * df['textcat_elderspeak_score']         

df[['elderspeak', 'textcat_elderspeak_score', 'reg_model_praat', 'reg_model_tekst', 'reg_model_volledig']]

,elderspeak,textcat_elderspeak_score,reg_model_praat,reg_model_tekst,reg_model_volledig
0,0,1.492628e-10,0.470329,0.359381,0.127765
1,1,5.497069e-06,0.740600,0.429100,0.423556
2,0,6.963545e-07,0.308622,0.363723,0.089986
3,0,2.806736e-05,0.180474,0.717499,0.212633
4,0,7.241415e-06,0.467125,0.362286,0.131557
5,1,6.058949e-05,0.205637,0.278221,0.158393
6,0,9.893542e-10,0.505150,0.267387,0.141166
7,0,1.449370e-02,0.492853,0.429835,0.156388
8,1,1.946689e-03,0.588510,0.431145,0.339568
9,1,9.999999e-01,0.641194,0.581487,1.087465


---

In [37]:
def cohend(d1: pd.Series, d2: pd.Series) -> float:
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)

    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)

    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))

    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)

    # the effect size
    return (u1 - u2) / s
    

In [38]:
elderspeak = df[(df['elderspeak'] == 1)]
not_elderspeak = df[(df['elderspeak'] == 0)]

cd_dict = {
    'kenmerk': [],
    'd': [],
    'effect': []
}

for c in ['spraaksnelheid', 'geluidsniveau', 'toonhoogte', 'cilt', 'woordlengteratio', 'aantal_collectieve_voornaamwoorden', 'aantal_verkleinwoorden', 'aantal_herhalingen', 'textcat_elderspeak_score', 'leeftijd', 'reg_model_praat', 'reg_model_tekst', 'reg_model_volledig']:
    d = cohend(elderspeak[c], not_elderspeak[c])
    abs_d = np.abs(d)
    
    if abs_d <= 0.01:
        effect = 'zeer klein'
    elif abs_d <= 0.2:
        effect = 'klein'
    elif abs_d <= 0.5:
        effect = 'middelmatig'
    elif abs_d <= 0.8:
        effect = 'groot'
    elif abs_d <= 1.2:
        effect = 'zeer groot'
    else:
        effect = 'reusachtig'

    cd_dict['kenmerk'].append(c) 
    cd_dict['d'].append(d) 
    cd_dict['effect'].append(effect) 

df_cd = pd.DataFrame.from_dict(cd_dict)
df_cd

,kenmerk,d,effect
0,spraaksnelheid,-0.061360,klein
1,geluidsniveau,0.412075,middelmatig
2,toonhoogte,0.488279,middelmatig
3,cilt,0.190854,klein
4,woordlengteratio,0.227493,middelmatig
5,aantal_collectieve_voornaamwoorden,-0.109957,klein
6,aantal_verkleinwoorden,0.092747,klein
7,aantal_herhalingen,0.353327,middelmatig
8,textcat_elderspeak_score,1.872398,reusachtig
9,leeftijd,0.322459,middelmatig
